<a href="https://colab.research.google.com/github/aakashaliraza/aakashaliraza/blob/main/JYN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -Uq langchain==0.3.13 langchain-google-genai python-dotenv

import os
import sqlite3
from google.colab import userdata
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Database
db_path = "conversation_history.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_input TEXT,
        assistant_response TEXT
    )
''')
conn.commit()

# Fetch API key
api_key = os.getenv("GOOGLE_API_KEY") or userdata.get("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Please set the environment variable or in userdata.")

# Initialize LLM and conversation chain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", api_key=api_key, temperature=0.7, max_output_tokens=256)
memory = ConversationBufferMemory()

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["history", "input"],
    template="You are a helpful chat assistant. You can access past conversation history from the database.\n{history}User: {input}\nAssistant:"
)

conversation = ConversationChain(llm=llm, memory=memory, prompt=prompt_template)

# Main conversation loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    try:
        # Generate response and save to DB
        response = conversation.predict(input=user_input)
        print(f"Assistant: {response}")
        cursor.execute("INSERT INTO conversations (user_input, assistant_response) VALUES (?, ?)", (user_input, response))
        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")

# Print conversation history
cursor.execute("SELECT user_input, assistant_response FROM conversations")
history_from_db = cursor.fetchall()
print("\nFull conversation history (from database):")
for user_input, assistant_response in history_from_db:
    print(f"User: {user_input}\nAssistant: {assistant_response}")

conn.close()


You: WHO ARE YOU?
Assistant: I am a large language model, trained by Google.

You: WHAT CAN YOU DO?
Assistant: I can provide information on a wide range of topics, answer questions, generate different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc., and translate languages. I am still under development and learning new things every day.

You: CAN THESE ANSWERS BE RELIABLE?
Assistant: While I strive to provide accurate and helpful information, it's important to remember that I am still under development and learning. My responses are based on the data I have been trained on, and there may be instances where my information is incomplete, outdated, or even incorrect. Therefore, it's always a good idea to double-check information, especially when it comes to important decisions or critical matters. Think of me as a helpful tool, but not a replacement for expert knowledge or your own critical thinking.

You: CAN THESE BE USED AS REFERENCE IN RESEARCH?
